In [1]:
import tensorflow as tf

In [2]:
import keras

Using TensorFlow backend.


In [3]:
import numpy as np
import os
import csv
from PIL import Image

import keras
import keras.preprocessing.image as img
from keras.applications import ResNet50, VGG16
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Flatten
from keras.layers.pooling import GlobalMaxPool2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras import backend as K

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
# get all the filenames
all_files = []
for path, subdirs, files in os.walk('data'):
    for name in files:
        all_files.append(os.path.join(path, name))


In [5]:
for f in all_files:
    temp_img = Image.open(f)
    temp_img = temp_img.crop((0, 600-435, 800, 600-435+185))
    temp_img.save('data_crop' + f.split('data')[1])


In [6]:
train_files = []
for path, subdirs, files in os.walk('data_crop\data\train '):
    for name in files:
        train_files.append(os.path.join(path, name))

In [7]:
np.random.shuffle(train_files)

valid_files = train_files[:500]
train_files = train_files[500:]

for f in valid_files:
    os.rename(f, 'data\valid ' + f.split('data\train ')[1])

In [8]:
# %mv data\valid\positive\* data\train\positive\
# %mv data\valid\negative\* data\train\negative\

In [9]:
def imagenet_mean(x):
    x = x[..., ::-1]
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

In [10]:
train_gen = img.ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.05,
    height_shift_range=0.1,
    preprocessing_function=imagenet_mean
)
test_gen = img.ImageDataGenerator(
    preprocessing_function=imagenet_mean
)


In [11]:
batch_size=64
img_size = (300,300)

In [12]:
train_batches = train_gen.flow_from_directory(
    r'data_crop\data\train',
    batch_size=batch_size,
    target_size = img_size,
    class_mode='binary'
)

valid_batches = test_gen.flow_from_directory(
    r'data_crop\data\valid',
    batch_size=batch_size,
    target_size = img_size,
    shuffle=False,
    class_mode='binary'
)

test_batches = test_gen.flow_from_directory(
    r'data_crop\data\test',
    batch_size=batch_size,
    target_size = img_size,
    shuffle=False,
    class_mode='binary'
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [13]:
temp_train_batch = train_batches.next()
print('X shape: ', temp_train_batch[0].shape)
print('Y shape: ', temp_train_batch[1].shape)

plt.imshow(temp_train_batch[0][0].astype('uint8'))

X shape:  (0, 300, 300, 3)
Y shape:  (0,)


IndexError: index 0 is out of bounds for axis 0 with size 0